In [1]:
from census import Census
from config import (census_api_key)
import pandas as pd
import requests
import json
import os
import gmaps
import csv

ModuleNotFoundError: No module named 'config'

In [235]:
url = "https://api.census.gov/data/2017/acs/acs5?get=B01003_001E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B01001I_001E,B09020_001E,B15003_002E,B15003_017E,B15003_022E,B15003_023E,B15003_025E,B17001_002E,B27001_002E,B27001_030E,NAME&for=place:*&key=" + census_api_key

response = requests.get(url)
response_json = response.json()

census_df = pd.DataFrame(response_json)

census_df.columns = census_df.iloc[0]
census_df = census_df[1:]

census_df.head()


,B01003_001E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,...,B15003_017E,B15003_022E,B15003_023E,B15003_025E,B17001_002E,B27001_002E,B27001_030E,NAME,state,place
1,174,0,0,0,0,0,0,0,0,0,...,0,0,0,0,45,69,105,"Abanda CDP, Alabama",01,00100
2,2594,51,40,35,60,40,29,8,27,109,...,567,112,112,4,500,1248,1164,"Abbeville city, Alabama",01,00124
3,4404,49,67,84,46,67,0,25,65,201,...,755,283,103,38,701,2063,2326,"Adamsville city, Alabama",01,00460
4,725,15,12,38,13,5,0,24,14,29,...,171,59,38,0,246,385,340,"Addison town, Alabama",01,00484
5,318,0,5,2,0,1,6,2,2,14,...,52,5,1,0,155,139,179,"Akron town, Alabama",01,00676


In [236]:
column_names = {"B01003_001E":"total_pop", "B01001_020E":"male_65_66", "B01001_021E":"male_67_69",
                "B01001_022E":"male_70_74", "B01001_023E":"male_75_79", "B01001_024E":"male_80_84",
                "B01001_025E":"male_over_85", "B01001_044E":"female_65_66", "B01001_045E":"female_67_69",
                "B01001_046E":"female_70_74", "B01001_047E":"female_75_79", "B01001_048E":"female_80_84",
                "B01001_049E":"female_over_85", "B02001_002E":"white_pop", "B02001_003E":"black_pop",
                "B02001_004E":"native_amer_pop", "B02001_005E":"asian_pop", "B02001_006E":"pac_island_pop",
                "B02001_007E":"other_race_pop", "B01001I_001E":"hispanic_pop", "B15003_002E":"no_high_school",
                "B15003_017E":"high_school_grad", "B15003_022E":"bachelor_deg", "B15003_023E":"master_deg", 
                "B15003_025E":"doctorate_deg", "B17001_002E":"below_poverty", "B27001_002E":"male_w_health_ins",
                "B27001_030E":"female_w_health_ins", "place":"city_FIPS"}

census_df.rename(columns=column_names, 
                 inplace=True)


In [252]:
for column in census_df.iloc[:, 0:29]:
    census_df[column] = pd.to_numeric(census_df[column])
    
census_df["with_health_ins"] = census_df["male_w_health_ins"] + census_df["female_w_health_ins"]
census_df.head()

,total_pop,male_65_66,male_67_69,male_70_74,male_75_79,male_80_84,male_over_85,female_65_66,female_67_69,female_70_74,...,doctorate_deg,below_poverty,male_w_health_ins,female_w_health_ins,NAME,state,city_FIPS,with_health_is,pop_over_65,with_health_ins
1,174,0,0,0,0,0,0,0,0,0,...,0,45,69,105,"Abanda CDP, Alabama",01,00100,174,NaN,174
2,2594,51,40,35,60,40,29,8,27,109,...,4,500,1248,1164,"Abbeville city, Alabama",01,00124,2412,NaN,2412
3,4404,49,67,84,46,67,0,25,65,201,...,38,701,2063,2326,"Adamsville city, Alabama",01,00460,4389,NaN,4389
4,725,15,12,38,13,5,0,24,14,29,...,0,246,385,340,"Addison town, Alabama",01,00484,725,NaN,725
5,318,0,5,2,0,1,6,2,2,14,...,0,155,139,179,"Akron town, Alabama",01,00676,318,NaN,318


In [289]:
census_df["city_FIPS"] = census_df["state"] + census_df["city_FIPS"]
census_df.head()

,total_pop,male_65_66,male_67_69,male_70_74,male_75_79,male_80_84,male_over_85,female_65_66,female_67_69,female_70_74,...,NAME,state,city_FIPS,with_health_is,pop_over_65,with_health_ins,with_degree,County,State,City
1,174,0,0,0,0,0,0,0,0,0,...,"Abanda CDP, Alabama",01,0100100,174,0,174,0,Abanda,Alabama,Abanda
2,2594,51,40,35,60,40,29,8,27,109,...,"Abbeville city, Alabama",01,0100124,2412,640,2412,228,Abbeville,Alabama,Abbeville
3,4404,49,67,84,46,67,0,25,65,201,...,"Adamsville city, Alabama",01,0100460,4389,814,4389,424,Adamsville,Alabama,Adamsville
4,725,15,12,38,13,5,0,24,14,29,...,"Addison town, Alabama",01,0100484,725,174,725,97,Addison,Alabama,Addison
5,318,0,5,2,0,1,6,2,2,14,...,"Akron town, Alabama",01,0100676,318,34,318,6,Akron,Alabama,Akron


In [290]:
census_df["pop_over_65"] = census_df.iloc[:,1:13].sum(axis=1)
census_df["with_degree"] = census_df.iloc[:, 23:26].sum(axis=1)

In [266]:
census_demographics_df=census_df[["total_pop", "with_health_is", "pop_over_65", "white_pop", "black_pop", "native_amer_pop",
                                "asian_pop", "pac_island_pop", "other_race_pop", "hispanic_pop", "no_high_school",
                                "high_school_grad", "with_degree", "below_poverty", "city_FIPS"]]

census_demographics_df.head()

,total_pop,with_health_is,pop_over_65,white_pop,black_pop,native_amer_pop,asian_pop,pac_island_pop,other_race_pop,hispanic_pop,no_high_school,high_school_grad,with_degree,below_poverty,city_FIPS
1,174,174,0,150,24,0,0,0,0,0,0,0,0,45,00100
2,2594,2412,640,1463,1083,0,0,0,14,117,34,567,228,500,00124
3,4404,4389,814,2077,2284,0,0,0,7,7,23,755,424,701,00460
4,725,725,174,685,0,3,0,0,0,0,5,171,97,246,00484
5,318,318,34,70,248,0,0,0,0,0,1,52,6,155,00676


In [278]:
for column in census_demographics_df.iloc[:, 1:14]:
    census_demographics_df[column] = census_demographics_df[column]/census_demographics_df.iloc[:,0]
census_demographics_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,total_pop,with_health_is,pop_over_65,white_pop,black_pop,native_amer_pop,asian_pop,pac_island_pop,other_race_pop,hispanic_pop,no_high_school,high_school_grad,with_degree,below_poverty,city_FIPS
1,174,1.000000,0.000000,0.862069,0.137931,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.258621,00100
2,2594,0.929838,0.246723,0.563994,0.417502,0.000000,0.0,0.0,0.005397,0.045104,0.013107,0.218581,0.087895,0.192753,00124
3,4404,0.996594,0.184832,0.471617,0.518619,0.000000,0.0,0.0,0.001589,0.001589,0.005223,0.171435,0.096276,0.159173,00460
4,725,1.000000,0.240000,0.944828,0.000000,0.004138,0.0,0.0,0.000000,0.000000,0.006897,0.235862,0.133793,0.339310,00484
5,318,1.000000,0.106918,0.220126,0.779874,0.000000,0.0,0.0,0.000000,0.000000,0.003145,0.163522,0.018868,0.487421,00676
